In [1]:
import os
import glob
from collections import OrderedDict

import numpy as np
import pandas as pd
import pickle as pkl
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.holtwinters import Holt



/home/rom/.local/lib/python3.6/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
%load_ext autoreload
%autoreload 1
%aimport summary.utils
# from summary import utils


In [3]:
# from pathlib import Path

# def de_norm(pred, criteria):
    
#     df_std = pd.read_pickle(os.path.join(Path(os.getcwd()).parents[0],'df_std.pkl'))
#     df_mean = pd.read_pickle(os.path.join(Path(os.getcwd()).parents[0],'df_mean.pkl'))
#     std = df_std[criteria]
#     mean = df_mean[criteria]
    
#     return (pred * std) + mean

# Load Data

In [3]:
# Data to play with

c_type = 'step'
imp = 'mice'
inter = 'nointer'
i= 0

base_dir = os.path.abspath(os.getcwd())

dirpath = os.path.join(base_dir, 'data')
y_test_30 = pd.read_pickle(os.path.join(dirpath, f"Y_{c_type}_30_test_{imp}_{inter}_{i}.pkl"))
X_test_30 = pd.read_pickle(os.path.join(dirpath, f"X_{c_type}_30_test_{imp}_{inter}_{i}.pkl"))

In [4]:
tr_dict = OrderedDict()
X_test_30['y_test_30']  = y_test_30['sys_30']
ids = X_test_30['hadm_id'].unique()

for idx in ids:
    tr_dict[idx] = X_test_30.loc[X_test_30.hadm_id == idx, 'y_test_30'].values

AttributeError: 'DataFrame' object has no attribute '_data'

## Normal Range

A naive step function implementation for samples that exceed the thersholds. For Sys thersholds are set to [90,180] to address hypotension and severe hypertension.

In [78]:
un = {}

for k,v in tr_dict.items():
    un[k] = utils.f_norm_naive_sys(v)

In [15]:
# get pred data

# file = open(os.path.join(os.getcwd(), 'scatters','30_mice.pkl'), 'rb')
with open(os.path.join(base_dir, dirpath, '30_mice.pkl'), 'rb') as file:
    sct = pkl.load(file)
ridge_test_30 = sct['30_bp_step_nointer_mice_ridge'][0]['pred_test']
xbg_test_30 = sct['30_bp_step_nointer_mice_ridge'][0]['pred_test']

ridge_test_30

array([156.03078887, 156.01701018, 151.05291772, ..., 157.80706512,
       157.58100312, 158.01372649])

In [96]:
pred_tr_dict = OrderedDict()
X = X_test_30.copy()
X['y_test_30']  = ridge_test_30

for idx in ids:
    pred_tr_dict[idx] = X.loc[X.hadm_id == idx, 'y_test_30'].values

In [97]:
# pred_trends_dict = utils.trend(pred_tr_dict)
# pred_trends_dict_new, distance_dict, res_dict = utils.get_trend(pred_tr_dict, True)


/home/bar/.local/lib/python3.6/site-packages/statsmodels/tsa/holtwinters/model.py:429: FutureWarning: After 0.13 initialization must be handled at model creation
  FutureWarning,
/home/bar/.local/lib/python3.6/site-packages/statsmodels/tsa/holtwinters/model.py:1439: RuntimeWarning: divide by zero encountered in log
  aic = self.nobs * np.log(sse / self.nobs) + k * 2
/home/bar/.local/lib/python3.6/site-packages/statsmodels/tsa/holtwinters/model.py:1444: RuntimeWarning: invalid value encountered in double_scalars
  aicc = aic + aicc_penalty
/home/bar/.local/lib/python3.6/site-packages/statsmodels/tsa/holtwinters/model.py:1445: RuntimeWarning: divide by zero encountered in log
  bic = self.nobs * np.log(sse / self.nobs) + k * np.log(self.nobs)


In [98]:
un_pred = {}

for k,v in pred_tr_dict.items():
    un_pred[k] = utils.f_norm_naive_sys(v)

# Normal Range Evaluation Measures

\begin{equation}
\begin{split}
    \frac{1}{n}\sum_{i}\max\left(0, (|f_{norm}\left(y_i\right)| - |f_{norm}\left(y^{pred}_i\right)\right|) + \max(0, -sign(f_{norm}(y_i) \cdot sign(f_{norm}(y^{pred}_i))\\
    \frac{1}{n}\sum_{i}\max\left(0, |f_{norm}\left(y^{pred}_i\right)| - |f_{norm}\left(y_i\right)\right|) + \max(0, -sign(f_{norm}(y_i)) \cdot sign(f_{norm}(y^{pred}_i))
\end{split}
\end{equation}

In [113]:
# consider making it per patient as well

print(f'missing important: {utils.f_norm_eval(un, un_pred)}')
print(f'false alarm: {utils.f_norm_eval(un_pred, un)}')


missing important: 0.0
false alarm: 0.8480776341104778


## Trend Utility

According to distance from Holt ES naive prediction.

In [77]:
trends_dict = utils.trend(tr_dict)
trends_dict_new, res_dict = utils.get_trend(tr_dict, True)

/home/bar/.local/lib/python3.6/site-packages/statsmodels/tsa/holtwinters/model.py:429: FutureWarning: After 0.13 initialization must be handled at model creation
  FutureWarning,
/home/bar/.local/lib/python3.6/site-packages/statsmodels/tsa/holtwinters/model.py:1439: RuntimeWarning: divide by zero encountered in log
  aic = self.nobs * np.log(sse / self.nobs) + k * 2
/home/bar/.local/lib/python3.6/site-packages/statsmodels/tsa/holtwinters/model.py:1444: RuntimeWarning: invalid value encountered in double_scalars
  aicc = aic + aicc_penalty
/home/bar/.local/lib/python3.6/site-packages/statsmodels/tsa/holtwinters/model.py:1445: RuntimeWarning: divide by zero encountered in log
  bic = self.nobs * np.log(sse / self.nobs) + k * np.log(self.nobs)


# Trend Based Evaluation


\begin{equation}
\begin{split}
    W_{g}(y_i)\cdot |y_i - y^{pred}_i|\\
    W_{g}(y^{pred}_i)\cdot |y_i - y^{pred}_i|
\end{split}
\end{equation}


In [ ]:
g_norm_true = utils.get_g_norm(tr_dict, trends_dict)
g_norm_pred = utils.get_g_norm(pred_tr_dict, trends_dict)

In [115]:
# Calc Weights

# holt_values = [v for k,v in trends_dict.items()]

# u_max = max(np.hstack(values))
# print(f'u_max: {u_max}')

# values = [v for k,v in pred_trends_dict.items()]
# u_max_pred = max(np.hstack(values))
# print(f'u_max_pred: {u_max_pred}')


# calculate weights
# weights = {k:v/u_max for k,v in trends_dict.items()}
# weights_pred = {k:v/u_max for k,v in pred_trends_dict.items()}

u_max: 1.6043447911875044
u_max_pred: 1.6043447911875044


### Consider checking if we need to denorm them before this code

In [131]:
w_trend_true, w_trend_pred = utils.w_trend(tr_dict, pred_tr_dict, g_norm_true, g_norm_pred)

# print(f'missing important: {w_trend(un, un_pred, weights)}')
# print(f'false alarm: {w_trend(un_pred, un, weights_pred)}')

print(f'missing important: {utils.mean_metric_score(w_trend_true)}')
print(f'false alarm: {utils.mean_metric_score(w_trend_pred)}')

missing important: 0.41214923001573267
false alarm: 0.41214923001573267


In [132]:
# pref_dict= {'Normal Range Deviation Miss':f_norm_eval(un, un_pred),\
#             'Normal Range Deviation False Alarm':f_norm_eval(un_pred, un),\
#            'Trend Deviation Miss': w_trend(un, un_pred, weights),\
#             'Trend Deviation False Alarm': w_trend(un_pred, un, weights_pred)}


pref_dict= {'Normal Range Deviation Miss':f_norm_eval(un, un_pred),\
            'Normal Range Deviation False Alarm':f_norm_eval(un_pred, un),\
           'Trend Deviation Miss': utils.w_trend_score(w_trend_true),\
            'Trend Deviation False Alarm':utils.w_trend_score(w_trend_pred)}

In [133]:
pd.DataFrame(pref_dict, index=[1]).round(3)

,Normal Range Deviation Miss,Normal Range Deviation False Alarm,Trend Deviation Miss,Trend Deviation False Alarm
1,0.0,0.848,0.412,0.412
